In [136]:
# Modules
import os
import csv
import codecs
import pandas as pd

In [137]:
# Get the current script file directory
thisFile_path = os.path.abspath("")
print(thisFile_path)

C:\Users\Arefin\Documents\UFT Data Analytics Bootcamp\Class folder\Analysis projects\Project 1


In [138]:
# Obtain GTA area categories file into a dataframe
csv_GTA_areas = os.path.join(thisFile_path, "GTA_by_areas.csv")
print(csv_GTA_areas)

C:\Users\Arefin\Documents\UFT Data Analytics Bootcamp\Class folder\Analysis projects\Project 1\GTA_by_areas.csv


In [139]:
GTA_areas_df = pd.read_csv(csv_GTA_areas)
GTA_areas_df.head()

,Durham Region,Halton Region,Peel Region,City of Toronto,York Region
0,Pickering,Halton hills,Caledon,Toronto,Vaughan
1,Ajax,Milton,Brampton,North York,Richmond Hill
2,Whitby,Oakville,Mississauga,Etobicoke,Markham
3,Oshawa,Burlington,NaN,Scarborough,Aurora
4,Uxbridge,NaN,NaN,NaN,Whitchurch-Stouffville


In [140]:
# Set path for the data csv file
csvpath_sch = os.path.join(thisFile_path, "School Rankings_interm_2018.csv")
print(csvpath_sch)

C:\Users\Arefin\Documents\UFT Data Analytics Bootcamp\Class folder\Analysis projects\Project 1\School Rankings_interm_2018.csv


In [142]:
with codecs.open(csvpath_sch, 'r', encoding='UTF-8', errors='ignore') as csvfile:
#with open(csvpath_sch, 'r', encoding='UTF-8') as csvfile:    
    csvreader = csv.reader(csvfile, delimiter=",") # read the file
    header = next(csvreader) # Isolate the header   
    data = [row for row in csvreader]  # Collect the data rows
    
# Convert csvreader object and header into a DataFrame
sch_df = pd.DataFrame.from_records(data, columns=header)
sch_df.head()

,Trend,School_Name,City,Current_rank,Overall_last,dummy_1,dummy_2,dummy_3
0,St,Michaels,Choir,Toronto,9.4,n/a,,
1,Abbey,Park,Oakville,9.2,n/a,,,
2,St,Robert,Thornhill,9.2,n/a,,,
3,Colonel,By,Gloucester,9.1,n/a,,,
4,St,Therese,of,Lisieux,Richmond,Hill,9.1,n/a


In [143]:
region_list = []
city_list = []
score_list = []
sch_name_list = []
keyword = "n/a"

# Main loop to iterate through each row of the SCHOOL dataframe
for index, row in sch_df.iterrows(): # This is gettig the current row index and the entire row from the school dataframe    
    row_values = [str(val) for val in row.values] # Convert row values to lowercase for case-insensitive search    
    curr_city = "" # initialize the current city as null space
    
    # Iterate through each cell of the row
    for col_index, cell in enumerate(row_values):
        exitCell_in_row_loop = False
        # Check if the cell contains the keyword
        if str(cell).lower() == keyword.lower():                        
            curr_city = row[col_index - 2] # Obtain current city            
            # Here we scan the GTA_by_areas dataframe to find out which region this curr_city belong to
            for region_name, region_data in GTA_areas_df.iteritems(): # obtain the region name (column name) and the cities under it
                exit_regName_loop = False # Initialize a flag to break the outer loop
                for index, city_name in region_data.iteritems():            
                    if city_name == curr_city:  #in row_values: .lower()
                        city_list.append(curr_city) # Collect current city
                        #print("City Name: ", city_name)
                        region_list.append(region_name)
                        score = row_values[col_index - 1] # Obtain score
                        score_list.append(float(score))   # Collect score  
                        sch_name = ' '.join(row_values[0: col_index - 2]).replace(',', ' ') # Obtain school name
                        sch_name_list.append(sch_name) # Collect school name            
                        # Loop Exit sequences
                        exit_regName_loop = True
                        exitCell_in_row_loop = True
                        break # exit out of city_name loop                        
                if exit_regName_loop == True: # break out of the region_name loop
                    break
        if exitCell_in_row_loop == True:
            break
            
    #break
   

sch_score_summary = pd.DataFrame({"Region Name":region_list,
                                  "City Name": city_list,
                                  "School Name": sch_name_list,
                                  "School Score": score_list
})
sch_score_summary.head(20)

C:\Users\Arefin\AppData\Local\Temp\ipykernel_18240\2985109539.py:19: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for region_name, region_data in GTA_areas_df.iteritems(): # obtain the region name (column name) and the cities under it
C:\Users\Arefin\AppData\Local\Temp\ipykernel_18240\2985109539.py:21: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, city_name in region_data.iteritems():


,Region Name,City Name,School Name,School Score
0,City of Toronto,Toronto,St Michaels Choir,9.4
1,Halton Region,Oakville,Abbey Park,9.2
2,York Region,Thornhill,St Robert,9.2
3,City of Toronto,Toronto,Ursula Franklin,9.1
4,York Region,Markham,Markville,8.9
5,Halton Region,Oakville,Oakville Trafalgar,8.9
6,Peel Region,Caledon,Wali ul Asr,8.9
7,York Region,Unionville,Unionville,8.8
8,York Region,Markham,St Augustine,8.8
9,York Region,Markham,Pierre Elliott Trudeau,8.7


In [144]:
# Generate csv file with clean DF for a desired year
GTA_school_score_csv = os.path.join(thisFile_path, "GTA school score clean DF 2018.csv")

# Write the DataFrame to CSV
sch_score_summary.to_csv(GTA_school_score_csv, index=False)